In [1]:
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable
from dfba.plot.matplotlib import *
import numpy as np
import math

from IPython.display import Image
from IPython.core.display import HTML

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/MoGeoModel"

In [3]:
model = read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [4]:
#Desired ratio of max growth, chosen from PPP:
ratio = 0.5

XFlux = model.slim_optimize()*ratio

with model:
    model.reactions.get_by_id("EX_Biomass_e").bounds = XFlux, XFlux
    model.objective = "EX_act_e"
    PFlux = model.slim_optimize()

set_flux = model.problem.Constraint(
    XFlux * model.reactions.EX_act_e.flux_expression - PFlux * model.reactions.EX_Biomass_e.flux_expression,
    lb=0,
    ub=0)
#model.add_cons_vars(set_flux)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
10fthf_e,EX_10fthf_e,4.711E-05,20,0.00%
ac_e,EX_ac_e,10,2,99.95%
adocbl_e,EX_adocbl_e,2.626E-05,72,0.01%
btn_e,EX_btn_e,2.356E-07,10,0.00%
ca2_e,EX_ca2_e,0.000371,0,0.00%
fe2_e,EX_fe2_e,2.626E-05,0,0.00%
fe3_e,EX_fe3_e,0.004005,0,0.00%
h_e,EX_h_e,9.046,0,0.00%
k_e,EX_k_e,0.08341,0,0.00%
mg2_e,EX_mg2_e,0.01154,0,0.00%


In [5]:
model.solver = "glpk"
dfba_model = DfbaModel(model)

In [6]:
X = KineticVariable("Biomass")#, initial_condition=C2_X_het_0)
Actt = KineticVariable("Acetate")#, initial_condition=C2_Actt_0)
Actn = KineticVariable("Acetone")#, initial_condition=C2_Actn_0)
O2 = KineticVariable("Oxygen")#, initial_condition=C2_O2_0)
CO2 = KineticVariable("CarbonDioxide")#, initial_condition=C2_CO2_0)

# add kinetic variables to dfba_model
dfba_model.add_kinetic_variables([X, Actt, Actn, O2, CO2])

# instances of ExchangeFlux
µ_het = ExchangeFlux("EX_Biomass_e")
q_het_Actt = ExchangeFlux("EX_ac_e")
q_het_Actn = ExchangeFlux("EX_act_e")
q_het_O2 = ExchangeFlux("EX_o2_e")
q_het_CO2 = ExchangeFlux("EX_co2_e")

dfba_model.add_initial_conditions(
    {
        "Biomass": 1,
        "Acetate": 10,
        "Acetone": 0,
        "Oxygen": 10,
        "CarbonDioxide": 0,
    }
)

# add exchange fluxes to dfba_model
dfba_model.add_exchange_fluxes([µ_het,q_het_Actt,q_het_Actn,q_het_O2,q_het_CO2])

# add rhs expressions for kinetic variables in dfba_model
dfba_model.add_rhs_expression("Biomass", -µ_het * X)
dfba_model.add_rhs_expression("Acetate", -q_het_Actt * 60 * X / 1000.0)
dfba_model.add_rhs_expression("Acetone", -q_het_Actn * 58 * X / 1000.0)
dfba_model.add_rhs_expression("Oxygen", -q_het_O2 * 32 * X / 1000.0 + 10)
dfba_model.add_rhs_expression("CarbonDioxide", -q_het_O2 * 44 * X / 1000.0)

# add lower/upper bound expressions for exchange fluxes in dfba_model together
# with expression that must be non-negative for correct evaluation of bounds
dfba_model.add_exchange_flux_lb("EX_ac_e", 10.5 * (Actt / (0.0027 + Actt)), Actt)
dfba_model.add_exchange_flux_lb("EX_o2_e", 15.0 * (O2 / (0.024 + O2)), O2)

# simulate model across interval t = [0.0,25.0](hours) with outputs for plotting
# every 0.1h and optional list of fluxes
concentrations, trajectories = dfba_model.simulate(
    0.0, 25.0, 0.1, ["EX_ac_e", "EX_act_e"]#, "EX_etoh(e)"]
)

In [7]:
concentrations

,time,Acetate,Acetone,Biomass,CarbonDioxide,Oxygen
0,0.0,10.000000,0.0,1.000000,0.000000,10.000000
1,0.1,9.998427,0.0,1.001394,0.065895,11.047924
2,0.2,9.996852,0.0,1.002789,0.131896,12.095924
3,0.3,9.995275,0.0,1.004186,0.197999,13.143999
4,0.4,9.993696,0.0,1.005586,0.264204,14.192148
...,...,...,...,...,...,...
245,24.5,9.540863,0.0,1.406819,19.274547,269.017853
246,24.6,9.538650,0.0,1.408780,19.367450,270.085418
247,24.7,9.536435,0.0,1.410743,19.460482,271.153078
248,24.8,9.534216,0.0,1.412709,19.553641,272.220830
